# **LoRA Finetuning of Mistral 7B-Instruct**
### Following are the steps:
- Install required libraries: `transformers`, `peft`, `accelerate`, `bitsandbytes`, `datasets`  
- Load model in 4-bit using `BitsAndBytesConfig` with NF4 dtype  
- Configure LoRA targeting `q_proj` and `v_proj` layers  
- Prepare dataset using Mistral's `[INST]` prompt template  
- Set training arguments: batch size 1, gradient accumulation, low learning rate  
- Enable gradient checkpointing to save memory  
- Initialize trainer with LoRA config and start training  
- Monitor GPU usage during training  
- Merge adapters into base model after training  
- Test model outputs for quality and safety  
- Deploy with appropriate disclaimers and safeguards

In [4]:
!pip install transformers accelerate  peft -U bitsandbytes -U datasets torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 fastai==2.7.19

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install -U bitsandbytes

In [2]:
!pip install -q huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TrainingArguments, Trainer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel, PeftConfig
from datasets import load_dataset
import torch
import os

In [4]:
#quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True , # Enables CPU offloading in float32
    bnb_4bit_quant_type="nf4"  # This fixes the fp4 on CPU error
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",  # Automatically places layers on GPU/CPU as needed
    quantization_config=quantization_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
#step 1: load dataset
dataset = load_dataset("empathetic_dialogues")

# Step 2: Save train split
train_dataset = dataset["train"]

print(train_dataset)

model = prepare_model_for_kbit_training(model)

README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

The repository for empathetic_dialogues contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/empathetic_dialogues.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

Dataset({
    features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
    num_rows: 76673
})


In [10]:
# to check the needed col-> 'utterance' is required for training
print(train_dataset[0])
print(train_dataset[1])
print(train_dataset[2])

{'conv_id': 'hit:0_conv:1', 'utterance_idx': 1, 'context': 'sentimental', 'prompt': 'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.', 'speaker_idx': 1, 'utterance': 'I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.', 'selfeval': '5|5|5_2|2|5', 'tags': ''}
{'conv_id': 'hit:0_conv:1', 'utterance_idx': 2, 'context': 'sentimental', 'prompt': 'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.', 'speaker_idx': 0, 'utterance': 'Was this a friend you were in love with_comma_ or just a best friend?', 'selfeval': '5|5|5_2|2|5', 'tags': ''}
{'conv_id': 'hit:0_conv:1', 'utterance_idx': 3, 'context': 'sentimental', 'prompt': 'I remember going to the fireworks with my best friend. There was a lot

In [6]:
# ftn to tokenize i.e. extract the utterance column
def tokenize(sample):
    prompt = f"{sample['utterance']}\n"
    encoding = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    encoding["labels"] = encoding["input_ids"].copy()
    return encoding

In [8]:
tokenized_dataset = train_dataset.map(tokenize, batched=False)

Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

In [10]:
# lora config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [12]:
training_args = TrainingArguments(
    output_dir="./mistral-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=100,  # For testing. Increase this.
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-12-2154013929>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, *

Step,Training Loss
10,8.355300
20,0.172900
30,0.138200
40,0.135000
50,0.114200
60,0.112100
70,0.114800
80,0.108700
90,0.112800
100,0.102100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=100, training_loss=0.9466073095798493, metrics={'train_runtime': 1654.5441, 'train_samples_per_second': 0.484, 'train_steps_per_second': 0.06, 'total_flos': 1.7491908624384e+16, 'train_loss': 0.9466073095798493, 'epoch': 0.010433784594517046})

In [13]:
model.save_pretrained("mistral-7b-lora-empathetic")
tokenizer.save_pretrained("mistral-7b-lora-empathetic")

('mistral-7b-lora-empathetic/tokenizer_config.json',
 'mistral-7b-lora-empathetic/special_tokens_map.json',
 'mistral-7b-lora-empathetic/chat_template.jinja',
 'mistral-7b-lora-empathetic/tokenizer.model',
 'mistral-7b-lora-empathetic/added_tokens.json',
 'mistral-7b-lora-empathetic/tokenizer.json')

In [26]:
#testing
# Load base Mistral
tokenizer = AutoTokenizer.from_pretrained("mistral-7b-lora-empathetic")

# Load LoRA adapter
peft_config = PeftConfig.from_pretrained("mistral-7b-lora-empathetic")

model = PeftModel.from_pretrained(
    model=model,
    model_id="mistral-7b-lora-empathetic"
)
model.eval()

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): MistralForCausalLM(
          (model): MistralModel(
            (embed_tokens): Embedding(32000, 4096)
            (layers): ModuleList(
              (0-31): 32 x MistralDecoderLayer(
                (self_attn): MistralAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
      

In [31]:
# testing
prompt = "I lost my job today."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
output = model.generate(inputs["input_ids"], max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I lost my job today. I was let go due to budget cuts. I’m feeling a mix of emotions right now. I’m sad, scared, and angry. I’m sad because I enjoyed my job and the people I worked with. I’m scared because I don’t know what the future holds. I’m angry because I feel like I was betrayed by my employer. I know they didn’t do this to intentionally hurt me, but it still feels like a personal attack.


In [32]:
merged_model = model.merge_and_unload()
merged_model.save_pretrained("mistral-merged")
tokenizer.save_pretrained("mistral-merged")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('mistral-merged/tokenizer_config.json',
 'mistral-merged/special_tokens_map.json',
 'mistral-merged/chat_template.jinja',
 'mistral-merged/tokenizer.model',
 'mistral-merged/added_tokens.json',
 'mistral-merged/tokenizer.json')